# Pt Med Claim Cnts
Counts of patient medical claims per year and quarter.  These are typicaly used to determine a patient's stability over a time period.

**Script**
* [scripts/pld/pt_med_claim_cnts.ipynb](./scripts/pld/pt_med_claim_cnts.ipynb)

**Prior Script(s)**
* [scripts/de/raven_header.ipynb](./scripts/de/raven_header.ipynb)

**Parameters**
* none

**Input**
* `coh_pt`
* `de_raven_header`
  
**Output**  
* `pld_med_cnt`
* `pld_med_cnt_pivot`

**Review**
* [scripts/pld/pt_med_claim_cnts.html](./scripts/pld/pt_med_claim_cnts.html)

In [ ]:
#Import libraries for this notebook
import pandas as pd  
from drg_connect import Snowflake
import numpy as np
import pickle
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Load connection variables to connect_dict
with open('../../out/conn/connect_dict.pickle', 'rb') as handle:
    connect_dict = pickle.load(handle)

#Create Eegine to connect to snowflake
snow = Snowflake(role=connect_dict['role'],
                 warehouse=connect_dict['warehouse'],
                 database=connect_dict['database'],
                 schema=connect_dict['schema'])

#Finish engine setup
engine = snow.engine
%load_ext sql_magic
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications


# Year Quarter Counts
Determine the counts of claims by year and quarter per patient

In [ ]:
%%read_sql
--Counts of claims by year and quarter for each patient
DROP TABLE IF EXISTS pld_med_cnt;
CREATE TRANSIENT TABLE pld_med_cnt AS 
  SELECT patient_id, 
         'cnt_' || Year(year_of_service)                                     AS year, 
         'cnt_' || Year(year_of_service) || '_Q' || Quarter(year_of_service) AS quarter, 
         Count(DISTINCT patient_id, year_of_service)                         AS med_claim_cnt 
    FROM de_raven_header
   GROUP BY patient_id, 
            year, 
            quarter; 

In [ ]:
%%read_sql
--Create unpivoted table with clean claim data
DROP TABLE IF EXISTS pld_med_cnt_unpivot;
CREATE TRANSIENT TABLE pld_med_cnt_unpivot AS
    SELECT patient_id,
           Right(year,4) AS period,
           Sum(med_claim_cnt) AS med_claim_cnt
      FROM pld_med_cnt
      GROUP BY patient_id, period
     UNION
    SELECT patient_id,
           Right(quarter,7) AS period,
           Sum(med_claim_cnt) AS med_claim_cnt
      FROM pld_med_cnt
     GROUP BY patient_id, period;

# Pivot Year
Pivot on the year to get the count of claims per year for each patient

In [ ]:
#Identify unique years to put in the dynamic sql
unique_values = snow.select("SELECT DISTINCT year FROM pld_med_cnt ORDER BY year")
yr_pivot_values = ",".join("'" + x +"'" for x in unique_values.values.flatten())
yr_pivot_values2 = ",".join(x for x in unique_values.values.flatten())
yr_pivot_values
yr_pivot_values2

In [ ]:
%%read_sql
--Create subset of counts by year
CREATE OR REPLACE TEMP TABLE tmp_pivot_yr AS
    SELECT patient_id,
           year,
           med_claim_cnt
     FROM pld_med_cnt;

--Pivot on year to get counts
CREATE OR REPLACE TEMP TABLE pld_med_cnt_yr AS
    SELECT *
      FROM tmp_pivot_yr
           PIVOT(Sum(med_claim_cnt) for year IN ({yr_pivot_values}))
           AS p (patient_id, {yr_pivot_values2}); 

# Pivot Quarter
Get claims counts by quarter for each patient

In [ ]:
#Determine the unique quarters to pivot and use dynamically later in queries
unique_values = snow.select("SELECT DISTINCT quarter FROM pld_med_cnt ORDER BY quarter")
qtr_pivot_values = ",".join("'" + x +"'" for x in unique_values.values.flatten())
qtr_pivot_values2 = ",".join(x for x in unique_values.values.flatten())
qtr_pivot_values
qtr_pivot_values2

In [ ]:
%%read_sql
--CREATE TRANSIENT TABLE subset with claim counts by quarter
CREATE OR REPLACE TEMP TABLE tmp_pivot_qtr AS
    SELECT patient_id,
           quarter,
           med_claim_cnt
     FROM pld_med_cnt;

--Pivot on quarter
CREATE OR REPLACE TEMP TABLE pld_med_cnt_qtr AS
    SELECT *
      FROM tmp_pivot_qtr
           PIVOT(Sum(med_claim_cnt) for quarter IN ({qtr_pivot_values}))
           AS p (patient_id, {qtr_pivot_values2}); 

# Pivot Counts
Pivot the data to get the final counts

In [ ]:
%%read_sql
DROP TABLE IF EXISTS pld_med_cnt_pivot;
CREATE TRANSIENT TABLE pld_med_cnt_pivot AS
    SELECT coh.patient_id,
           {yr_pivot_values2},
           {qtr_pivot_values2}
      FROM coh_pt coh
           LEFT JOIN pld_med_cnt_yr yr
                  ON coh.patient_id = yr.patient_id
           LEFT JOIN pld_med_cnt_qtr qtr
                  ON coh.patient_id = qtr.patient_id

# Remove "cnt_"
remove "cnt_" from the 